In [1]:
import trimesh
import numpy as np
from utils import *
import open3d as o3d

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
def pymesh_to_trimesh(mesh):
    return trimesh.Trimesh(vertices=mesh.vertices, faces=mesh.faces)

def trimesh_to_pymesh(mesh):
    return pymesh.form_mesh(vertices=mesh.vertices, faces=mesh.faces)

def compute_median_edge_length(mesh):
    face_indices = mesh.faces 
    verts = mesh.vertices

    edge_lengths = []
    unique_edges = set()


    for f in face_indices:
        # Each face has 3 edges: (f[0], f[1]), (f[1], f[2]), (f[2], f[0])
        edges = [(f[0], f[1]), (f[1], f[2]), (f[2], f[0])]
        
        for edge in edges:
            edge = tuple(sorted(edge))
            if edge not in unique_edges:
                unique_edges.add(edge)
                
                # Calculate the edge length
                e_length = np.linalg.norm(verts[edge[0]] - verts[edge[1]])
                edge_lengths.append(e_length)

    return np.median(edge_lengths) if edge_lengths else 0

def collapse_long_edges(mesh):
    tol = compute_median_edge_length(mesh)
    new_mesh, _ = pymesh.split_long_edges(mesh, tol)
    return new_mesh

def count_num_components(mesh):
    mesh = pymesh_to_trimesh(mesh)
    components = mesh.split(only_watertight=True)
    return len(components)

In [ ]:
mesh = pymesh.load_mesh("data/two_spheres2.ply")

In [15]:
count = count_num_components(mesh)
intersecting_faces = pymesh.detect_self_intersection(mesh)
intersected_faces = set(intersecting_faces.flatten())

remaining_faces = np.setdiff1d(np.arange(mesh.num_faces), list(intersected_faces))
remaining_faces_vertices = mesh.faces[remaining_faces]

unique_vertices, new_faces = np.unique(remaining_faces_vertices, return_inverse=True)
new_mesh = pymesh.form_mesh(mesh.vertices[unique_vertices], new_faces.reshape(-1, 3))


In [16]:
submeshes = pymesh_to_trimesh(new_mesh).split(only_watertight=False)
submeshes_sorted = sorted(submeshes, key=lambda x: len(x.vertices), reverse=True)
submeshes_needed = submeshes_sorted[:count]

In [17]:
repaired_submesh_needed = []
for submesh in submeshes_needed:
    submesh = pymesh.convex_hull(trimesh_to_pymesh(submesh))
    submesh = collapse_long_edges(submesh)
    submesh = pymesh_to_trimesh(submesh)
    repaired_submesh_needed.append(submesh)

In [18]:
final = trimesh.util.concatenate(repaired_submesh_needed)

In [19]:
visualize(final)

Widget(value='<iframe src="http://localhost:40139/index.html?ui=P_0x7efd123a2bf0_3&reconnect=auto" class="pyvi…

None

In [13]:
visualize(repaired_submesh_needed[0])

Widget(value='<iframe src="http://localhost:40139/index.html?ui=P_0x7efd35d7f8b0_2&reconnect=auto" class="pyvi…

None